In [2]:
import rawpy
import cv2
import numpy as np
from multiprocessing.pool import Pool
import utils
import os



ModuleNotFoundError: No module named 'cv2'

In [ ]:
#Instantiation
lum_downsample=utils.Downsampling(ratio='4:4:0')
chr_downsample=utils.Downsampling(ratio='4:2:0')
image_block=utils.ImageBlock(block_height=8,block_width=8)
dct2d=utils.DCT2D(norm='ortho')
quantization=utils.Quantization()

In [ ]:
#Read raw images
path='Images/Original.ARW'
raw=rawpy.imread(path)


#Postprocess the currently loaded RAW image and return the new resulting image as numpy array.

rgb_img=raw.postprocess()


#Changing the colorspace from RGB to YCbCr
ycc_img=cv2.cvtColor(rgb_img,cv2.COLOR_XYZ2RGB)

#Center
ycc_img=ycc_img.astype(int)-128

In [ ]:
#Downsampling
Y = lum_downsample(ycc_img[:,:,0])
Cr = chr_downsample(ycc_img[:,:,1])
Cb = chr_downsample(ycc_img[:,:,2])
ycc_img = np.stack((Y, Cr, Cb), axis=2)



# Create 8X8 blocks
blocks,indices=image_block.forward(ycc_img)

In [ ]:
#Compression 

def process_block(block,index):
    #DCT
    encoded=dct2d.forward(block)
    
    if index[2]==0:
        channel_type='lum'
    else:
        channel_type='chr'
    
    #Quantization
    encoded=quantization.forward(encoded,channel_type)
    
    #Dequantization
    decoded=quantization.backward(encoded,channel_type)
    
    #Reverse DCT
    compressed=dct2d.backward(decoded)
    return compressed


compressed=np.array(Pool().starmap(process_block,zip(blocks,indices)))

In [ ]:
#Reconstruct images from blocks
ycc_img_compressed=image_block.backward(compressed,indices)

#Rescale
ycc_img_compressed=(ycc_img_compressed+128).astype('uint8')

#Transform back to RGB
rgb_img_compressed=cv2.cvtColor(ycc_img_compressed,cv2.COLOR_YCrCb2RGB)


Image.fromarray(rgb_img_compressed).save(os.path.join('images','result.jpeg'))